In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
exp_name = '2021_03_01_flying'; appendix="_30cm-glass";
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

In [ ]:
df_total

In [ ]:
import seaborn as sns

max_dist = 50 # cm
for i, row in df_total.iterrows():

    positions_cm = row.positions[:, :3] * 100
    
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[:, 0], y=positions_cm[:, 1], 
                    hue=row.seconds, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    axs[0].set_xlim(-max_dist/2, max_dist/2)
    axs[0].set_ylim(-max_dist/2, max_dist/2)
    axs[0].legend(loc='lower right')

    axs[1].plot(row.seconds, positions_cm[:, 0], label='x')
    axs[1].plot(row.seconds, positions_cm[:, 1], label='y')
    axs[1].plot(row.seconds, positions_cm[:, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(row.seconds, row.positions[:, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')

    plt.tight_layout()

# Analysis

In [ ]:
from frequency_analysis import add_spectrogram

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

print(df_total.appendix.unique())
print(df_total.source.unique())
print(df_total.distance.unique())

In [ ]:
mic_idx = 0
min_freq = 2900 
max_freq = 5000

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
fig2, ax_tot = plt.subplots()
fig2.set_size_inches(10, 5)
for i_col, row in df_total.iterrows():
    all_frequencies = np.unique(row.frequencies_matrix)
    spec = row.spectrogram[:, mic_idx, :]
    spec[spec == 0] = np.nan
    total = np.nanmean(spec, axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.suptitle(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    fig.set_size_inches(10, 5)
    ax.pcolorfast(row.seconds, 
                  all_frequencies, spec[:-1, :-1])
    ax.set_ylabel('frequency [Hz]')
    ax.set_ylim(min_freq, max_freq)
    
    ax_tot.plot(all_frequencies, total, color=f"C{i_col}", label=label)
ax_tot.legend(loc='upper right')
#ax_tot.set_ylim(0, 3)
ax_tot.set_yscale('log')
ax_tot.set_xlim(min_freq, max_freq)

In [ ]:
from wall_detector import WallDetector, get_probability_cost, get_probability_fft

fig, ax_slices = plt.subplots()
fig.set_size_inches(10, 5)
fig, ax_proba_cost = plt.subplots()
fig.set_size_inches(10, 5)
fig, ax_proba_fft = plt.subplots()
fig.set_size_inches(10, 5)

distances_grid = np.arange(10, 60)

for i_col, row in df_total.iterrows():
    wall_detector = WallDetector(exp_name=exp_name)
    spec_masked, freqs_masked = wall_detector.fill_from_row(row)
    
    spec = spec_masked[:, mic_idx, :]
    spec[spec == 0] = np.nan
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.suptitle(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    fig.set_size_inches(10, 5)
    ax.pcolorfast(row.seconds, 
                  freqs_masked, spec[:-1, :-1])
    ax.set_ylabel('frequency [Hz]')
    ax.set_ylim(min_freq, max_freq)
    
    wall_detector.remove_bad_freqs(verbose=True)
    wall_detector.remove_spurious_freqs(verbose=True)
    f_slice, freqs = wall_detector.get_frequency_slice()
    ax_slices.plot(freqs, f_slice[mic_idx, :], label=label)
    
    proba_cost = get_probability_cost(
        f_slice[mic_idx, :], freqs, distances_grid, mic_idx=mic_idx
    )
    distances_fft, proba_fft = get_probability_fft(f_slice[mic_idx, :], freqs)
    
    ax_proba_cost.plot(distances_grid, proba_cost, label=label, color=f'C{i_col}')
    ax_proba_fft.plot(distances_fft, proba_fft, label=label, color=f'C{i_col}', ls=':')
    
ax_proba_fft.set_xlim(min(distances_grid), max(distances_grid))

ax_slices.legend(loc='upper left')
ax_proba_fft.legend(loc='upper left')
ax_proba_cost.legend(loc='upper left')